In [53]:
# Load Perfetto Protos
import os
import sys
proto_python_path=os.path.join(os.path.abspath(os.curdir), "perfetto_protos")
print(proto_python_path)
sys.path.append(proto_python_path)

/Users/xl/Development/perfetto-example/perfetto_protos


In [54]:
from protos.perfetto.trace import trace_pb2
from protos.perfetto.trace import trace_packet_pb2
from protos.perfetto.trace.track_event import track_event_pb2

In [55]:
# Perfetto Tools
import jinja2
import gzip

def export_perfetto(trace, name, path="./reports"):
  if not os.path.exists(path):
      os.makedirs(path)
  trace_path = f"{path}/{name}.gz.pefetto"
  html_path = f"{path}/{name}.html"
  trace_file = os.path.basename(trace_path)

  # data
  compressed_data = gzip.compress(trace.SerializeToString())
  with open(trace_path, "wb") as fp:
    fp.write(compressed_data)

  # html
  with open("./index.html.template", "r") as fp:
    template = jinja2.Template(fp.read())
  with open(html_path, "w") as fp:
    fp.write(template.render(trace_file=trace_file))

  return html_path

In [56]:
# Trace Tools

def add_clock(timezone_delta=None):
  ret = [trace_packet_pb2.TracePacket(trace_packet_defaults=dict(timestamp_clock_id=1))] #REALTIME
  if timezone_delta is not None:
    offset_minutes = round(timezone_delta.total_seconds() // 60)
    ret.append(
      trace_packet_pb2.TracePacket(system_info=dict(timezone_off_mins=offset_minutes))
    )
  return ret

def add_track(name, *, uuid, parent_uuid=0, **args):
  return [
    trace_packet_pb2.TracePacket(
      track_descriptor=dict(
        name=name, uuid=uuid, parent_uuid=parent_uuid, **args
      )
    )
  ]


def add_scope(name, *, start, end, track_uuid, data=None, **args):
  return [
      trace_packet_pb2.TracePacket(
          trusted_packet_sequence_id=42,
          track_event=dict(
              type="TYPE_SLICE_BEGIN",
              timestamp_absolute_us=start,
              track_uuid=track_uuid,
              name=name,
              debug_annotations=(
                  None
                  if data is None
                  else [
                      dict(name=name, string_value=str(value))
                      for name, value in data.items()
                  ]
              ),
              **args,
          ),
      ),
      trace_packet_pb2.TracePacket(
          trusted_packet_sequence_id=42,
          track_event=dict(
              type="TYPE_SLICE_END",
              track_uuid=track_uuid,
              timestamp_absolute_us=end,
          ),
      ),
  ]


def add_instant_event(name, *, start, track_uuid, data=None, **args):
  return [
      trace_packet_pb2.TracePacket(
          trusted_packet_sequence_id=42,
          track_event=dict(
              type="TYPE_INSTANT",
              timestamp_absolute_us=start,
              track_uuid=track_uuid,
              name=name,
              debug_annotations=(
                  None
                  if data is None
                  else [
                      dict(name=name, string_value=str(value))
                      for name, value in data.items()
                  ]
              ),
              **args,
          ),
      )
  ]


In [57]:
# Example Perfetto

import datetime
base = round(datetime.datetime.now().now().timestamp() * 1e6)

trace = trace_pb2.Trace()

trace.packet.extend(add_clock(datetime.datetime.now().astimezone().utcoffset()))
# Top Section
trace.packet.extend(add_track("Server", uuid=1, child_ordering="LEXICOGRAPHIC"))

# Track
trace.packet.extend(add_track("Network", uuid=2, parent_uuid=1))

# Instant Events
trace.packet.extend(add_instant_event("Transfer Requested", start=base + 42, track_uuid=2, data={"user": "Hadi"}))
trace.packet.extend(add_instant_event("Transfer Requested", start=base + 142, track_uuid=2, data={"user": "Hadi"}))

# Nested Scopes: scopes in one track have to nested.
trace.packet.extend(add_scope("RPC", start=base + 50, end=base+ 200, track_uuid=2, data={"size": "220MB", "url": "http://xldrx.com/"}))
trace.packet.extend(add_scope("Wire", start=base + 100, end=base+150, track_uuid=2))

# Unnested Scopes: Unnested scopes need separate tracks (with same name)
trace.packet.extend(add_track("GPU", uuid=3, parent_uuid=1))
trace.packet.extend(add_track("GPU", uuid=4, parent_uuid=1))

trace.packet.extend(add_scope("Matmul", start=base+10, end=base+150, track_uuid=3))
trace.packet.extend(add_scope("Matmul", start=base+90, end=base+250, track_uuid=4))


# Flows
trace.packet.extend(add_track("Client", uuid=5))
trace.packet.extend(add_track("Thread-0", uuid=6, parent_uuid=5))
trace.packet.extend(add_track("Thread-1", uuid=7, parent_uuid=5))

trace.packet.extend(add_scope("Stage0", start=base+10, end=base+50, flow_ids=[1], track_uuid=6))
trace.packet.extend(add_scope("Stage1", start=base+100, end=base+120, flow_ids=[1], track_uuid=6))
trace.packet.extend(add_scope("Stage2", start=base+110, end=base+150, flow_ids=[1], track_uuid=7))
trace.packet.extend(add_scope("Stage3", start=base+160, end=base+250, flow_ids=[1], track_uuid=6))

export_perfetto(trace, "example")

'./reports/example.html'